# 🏋️ Coach Sportif Intelligent

Ce projet vise à :
- Recommander un programme d’entraînement personnalisé
- Prédire la progression des performances sportives

Les données sont simulées à partir de profils utilisateurs et journaux d’entraînement.

In [13]:
import pandas as pd

# Chargement des données simulées
df_reco = pd.read_csv("data_reco.csv")
df_perf = pd.read_csv("data_perf.csv")

df_reco.head()

,age,sexe,poids,taille,objectif,historique_sportif,programme_recommandé
0,56,H,60,159,endurance,débutant,cardio
1,46,F,78,166,endurance,avancé,musculation
2,32,H,85,169,perte de poids,avancé,cardio
3,25,H,74,173,endurance,avancé,HIIT
4,38,F,70,154,perte de poids,intermédiaire,HIIT


## 🔍 Partie 1 : Recommandation de programme d'entraînement
Nous utilisons un modèle de classification (KNN) pour prédire un type de programme selon le profil et l’objectif.

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Encodage des variables catégorielles
df_encoded = df_reco.copy()
for col in ["sexe", "objectif", "historique_sportif", "programme_recommandé"]:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])

X = df_encoded.drop("programme_recommandé", axis=1)
y = df_encoded["programme_recommandé"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_reco = KNeighborsClassifier(n_neighbors=5)
model_reco.fit(X_train, y_train)

y_pred = model_reco.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.18      0.24      0.21        17
           1       0.29      0.35      0.32        20
           2       0.21      0.13      0.16        23

    accuracy                           0.23        60
   macro avg       0.23      0.24      0.23        60
weighted avg       0.23      0.23      0.23        60



In [15]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

mlflow.set_experiment("CoachSportif")


2025/05/26 16:06:07 INFO mlflow.tracking.fluent: Experiment with name 'CoachSportif' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/Users/mazen/Downloads/COACH_SPORTIF/mlruns/542932965039613921', creation_time=1748268367219, experiment_id='542932965039613921', last_update_time=1748268367219, lifecycle_stage='active', name='CoachSportif', tags={}>

In [16]:
with mlflow.start_run(run_name="KNN - Recommandation"):
    mlflow.log_param("model", "KNN")
    mlflow.log_param("n_neighbors", 5)
    
    mlflow.sklearn.log_model(model_reco, "model_reco_knn")
    from sklearn.metrics import accuracy_score
    acc = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", acc)
    print("MLflow run KNN terminé avec accuracy :", acc)


2025/05/26 16:06:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MLflow run KNN terminé avec accuracy : 0.23333333333333334


In [17]:
import joblib
joblib.dump(model_reco, "model_reco.pkl")

['model_reco.pkl']

## 📈 Partie 2 : Prédiction de performance
Nous utilisons un modèle de régression (XGBoost) pour estimer la progression future (reps ou charge).

In [18]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

X = df_perf.drop("progression_future", axis=1)
y = df_perf["progression_future"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_perf = XGBRegressor()
model_perf.fit(X_train, y_train)

y_pred = model_perf.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))

MSE: 918.5921456106537


In [19]:
joblib.dump(model_perf, "model_perf.pkl")

['model_perf.pkl']

In [20]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import joblib

# Chargement
df = pd.read_csv("data_perf.csv")

X = df.drop("progression_future", axis=1)
y = df["progression_future"]

# Modèle
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model_perf = RandomForestRegressor()
model_perf.fit(X_train, y_train)

# Sauvegarde
joblib.dump(model_perf, "model_perf.pkl")


['model_perf.pkl']